# Lab 2-1: FastAPI 모델 서빙 - 모델 학습

## 📋 실습 개요

이 Notebook에서는 Iris 분류 모델을 학습하고 평가합니다.

**학습 내용:**
1. 데이터 로드 및 탐색 (EDA)
2. Train/Test 데이터 분할
3. RandomForest 모델 학습
4. 모델 평가 및 시각화
5. 모델 저장
6. 로컬 예측 테스트

**소요시간:** 30분

---

## Step 0: 환경 설정

필요한 패키지를 설치하고 import합니다.

In [ ]:
# 패키지 설치 (필요한 경우)
!pip install scikit-learn==1.5.2 numpy==1.26.4 pandas matplotlib seaborn joblib -q

print("✅ 패키지 설치 완료!")

In [ ]:
# 패키지 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("✅ Import 완료")

---

## Step 1: 데이터 로드 및 탐색

Iris 데이터셋을 로드하고 기본 통계를 확인합니다.

In [ ]:
# 데이터 로드
print("=" * 60)
print("  Step 1: 데이터 로드")
print("=" * 60)

iris = load_iris()
X = iris.data
y = iris.target

print(f"\n✅ 데이터 로드 완료")
print(f"   - 전체 샘플 수: {len(X)}")
print(f"   - 피처 수: {len(iris.feature_names)}")
print(f"   - 클래스 수: {len(iris.target_names)}")
print(f"   - 클래스 이름: {', '.join(iris.target_names)}")

In [ ]:
# DataFrame으로 변환
df = pd.DataFrame(X, columns=iris.feature_names)
df['species'] = pd.Categorical.from_codes(y, iris.target_names)

print("\n📊 데이터 샘플 (처음 5개):")
display(df.head())

print("\n📈 기본 통계:")
display(df.describe())

### 데이터 시각화

In [ ]:
# 클래스 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 클래스별 샘플 수
species_counts = df['species'].value_counts()
axes[0].bar(species_counts.index, species_counts.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0].set_title('Number of samples per class', fontsize=14, weight='bold')
axes[0].set_xlabel('Species')
axes[0].set_ylabel('Count')
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
axes[1].pie(species_counts.values, labels=species_counts.index, autopct='%1.1f%%',
            colors=['#FF6B6B', '#4ECDC4', '#45B7D1'], startangle=90)
axes[1].set_title('Class ratio', fontsize=14, weight='bold')

plt.tight_layout()
plt.show()

print("\n✅ 균형잡힌 데이터셋: 각 클래스당 50개 샘플")

In [ ]:
# 피처별 분포 (Pairplot)
print("\n🔍 피처 간 관계 시각화...")
sns.pairplot(df, hue='species', diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Iris Features Pairplot', y=1.02, fontsize=16, weight='bold')
plt.show()

print("\n💡 인사이트:")
print("   - petal length와 petal width가 클래스 구분에 가장 유용")
print("   - setosa는 다른 클래스와 명확히 구분됨")
print("   - versicolor와 virginica는 약간 겹침")

---

## Step 2: 데이터 분할

Train/Test 데이터로 분할합니다 (80:20 비율).

In [ ]:
print("=" * 60)
print("  Step 2: 데이터 분할")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # 클래스 비율 유지
)

print(f"\n✅ 데이터 분할 완료")
print(f"   - 학습 데이터: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   - 테스트 데이터: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

# 클래스별 분포 확인
print("\n📈 클래스별 분포:")
for i, class_name in enumerate(iris.target_names):
    train_count = np.sum(y_train == i)
    test_count = np.sum(y_test == i)
    print(f"   - {class_name:12s}: Train={train_count:2d}, Test={test_count:2d}")

---

## Step 3: 모델 학습

RandomForest 분류 모델을 학습합니다.

In [ ]:
print("=" * 60)
print("  Step 3: 모델 학습")
print("=" * 60)

print("\n🔄 RandomForest 모델 학습 중...")
print("   - n_estimators: 100")
print("   - random_state: 42")
print("   - n_jobs: -1 (모든 CPU 사용)")

model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

print("\n✅ 모델 학습 완료")
print(f"   - 트리 개수: {model.n_estimators}")
print(f"   - 피처 개수: {model.n_features_in_}")

---

## Step 4: 모델 평가

학습된 모델의 성능을 평가합니다.

In [ ]:
print("=" * 60)
print("  Step 4: 모델 평가")
print("=" * 60)

# 학습 데이터 평가
train_predictions = model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)

# 테스트 데이터 평가
test_predictions = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"\n📊 정확도 (Accuracy):")
print(f"   - 학습 데이터: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"   - 테스트 데이터: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

if train_accuracy - test_accuracy > 0.1:
    print(f"   ⚠️  과적합 가능성 (차이: {(train_accuracy-test_accuracy)*100:.2f}%)")
else:
    print(f"   ✅ 일반화 성능 양호")

### 피처 중요도 분석

In [ ]:
# 피처 중요도 계산
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🎯 피처 중요도:")
for idx, row in feature_importance.iterrows():
    bar = "█" * int(row['importance'] * 50)
    print(f"   {row['feature']:25s} {row['importance']:.4f} {bar}")

# 시각화
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], 
         color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#95E1D3'])
plt.xlabel('Importance', fontsize=12)
plt.title('Feature Importance', fontsize=14, weight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 혼동 행렬 (Confusion Matrix)

In [ ]:
# 혼동 행렬 계산
cm = confusion_matrix(y_test, test_predictions)

# 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.title('Confusion Matrix', fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n📋 혼동 행렬 (숫자):")
print(cm)

### 클래스별 성능

In [ ]:
# Classification report
report = classification_report(
    y_test, test_predictions,
    target_names=iris.target_names,
    output_dict=True
)

print("\n📈 클래스별 성능:")
for class_name in iris.target_names:
    metrics = report[class_name]
    print(f"   - {class_name:12s}: "
          f"Precision={metrics['precision']:.3f}, "
          f"Recall={metrics['recall']:.3f}, "
          f"F1-Score={metrics['f1-score']:.3f}")

# 시각화
metrics_df = pd.DataFrame({
    'Precision': [report[c]['precision'] for c in iris.target_names],
    'Recall': [report[c]['recall'] for c in iris.target_names],
    'F1-Score': [report[c]['f1-score'] for c in iris.target_names]
}, index=iris.target_names)

metrics_df.plot(kind='bar', figsize=(10, 6), rot=0, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
plt.title('Class-specific performance metrics', fontsize=14, weight='bold')
plt.ylabel('Score', fontsize=12)
plt.xlabel('Species', fontsize=12)
plt.ylim(0, 1.1)
plt.legend(loc='lower right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

---

## Step 5: 모델 저장

학습된 모델을 파일로 저장합니다.

In [ ]:
import os

print("=" * 60)
print("  Step 5: 모델 저장")
print("=" * 60)

model_path = "model.joblib"
print(f"\n💾 모델 저장 중: {model_path}")

joblib.dump(model, model_path)

# 파일 크기 확인
file_size = os.path.getsize(model_path)
if file_size < 1024:
    size_str = f"{file_size} bytes"
elif file_size < 1024 * 1024:
    size_str = f"{file_size / 1024:.2f} KB"
else:
    size_str = f"{file_size / (1024 * 1024):.2f} MB"

print(f"\n✅ 모델 저장 완료")
print(f"   - 파일명: {model_path}")
print(f"   - 파일 크기: {size_str}")
print(f"   - 저장 위치: {os.path.abspath(model_path)}")

---

## Step 6: 로컬 예측 테스트

저장된 모델을 로드하여 예측을 테스트합니다.

In [ ]:
print("=" * 60)
print("  Step 6: 로컬 예측 테스트")
print("=" * 60)

# 모델 로드
print(f"\n📂 모델 로드: {model_path}")
loaded_model = joblib.load(model_path)
print("✅ 모델 로드 성공")

### 테스트 케이스 1: Setosa

In [ ]:
# Setosa 샘플 (전형적인 값)
sample_setosa = np.array([[5.1, 3.5, 1.4, 0.2]])

prediction = loaded_model.predict(sample_setosa)[0]
probabilities = loaded_model.predict_proba(sample_setosa)[0]

print("\n🧪 테스트 1: Setosa")
print(f"   입력: {sample_setosa[0]}")
print(f"   예측: {iris.target_names[prediction]}")
print(f"   신뢰도: {probabilities[prediction]:.4f}")
print(f"   확률 분포:")
for i, class_name in enumerate(iris.target_names):
    print(f"      - {class_name}: {probabilities[i]:.4f}")

### 테스트 케이스 2: Versicolor

In [ ]:
# Versicolor 샘플
sample_versicolor = np.array([[5.9, 3.0, 4.2, 1.5]])

prediction = loaded_model.predict(sample_versicolor)[0]
probabilities = loaded_model.predict_proba(sample_versicolor)[0]

print("\n🧪 테스트 2: Versicolor")
print(f"   입력: {sample_versicolor[0]}")
print(f"   예측: {iris.target_names[prediction]}")
print(f"   신뢰도: {probabilities[prediction]:.4f}")
print(f"   확률 분포:")
for i, class_name in enumerate(iris.target_names):
    print(f"      - {class_name}: {probabilities[i]:.4f}")

### 테스트 케이스 3: Virginica

In [ ]:
# Virginica 샘플
sample_virginica = np.array([[6.7, 3.0, 5.2, 2.3]])

prediction = loaded_model.predict(sample_virginica)[0]
probabilities = loaded_model.predict_proba(sample_virginica)[0]

print("\n🧪 테스트 3: Virginica")
print(f"   입력: {sample_virginica[0]}")
print(f"   예측: {iris.target_names[prediction]}")
print(f"   신뢰도: {probabilities[prediction]:.4f}")
print(f"   확률 분포:")
for i, class_name in enumerate(iris.target_names):
    print(f"      - {class_name}: {probabilities[i]:.4f}")

### 배치 예측 테스트

In [ ]:
# 여러 샘플 동시 예측
samples = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Setosa
    [5.9, 3.0, 4.2, 1.5],  # Versicolor
    [6.7, 3.0, 5.2, 2.3]   # Virginica
])

predictions = loaded_model.predict(samples)
probabilities = loaded_model.predict_proba(samples)

print("\n🧪 배치 예측 테스트:")
for i, (sample, pred, probs) in enumerate(zip(samples, predictions, probabilities), 1):
    print(f"\n   샘플 {i}:")
    print(f"      입력: {sample}")
    print(f"      예측: {iris.target_names[pred]}")
    print(f"      신뢰도: {probs[pred]:.4f}")

---

## 🎉 완료!

### ✅ 완료된 작업

- [x] 데이터 로드 및 EDA
- [x] Train/Test 분할
- [x] RandomForest 모델 학습
- [x] 모델 평가 및 시각화
- [x] 피처 중요도 분석
- [x] 혼동 행렬 확인
- [x] 모델 저장 (`model.joblib`)
- [x] 로컬 예측 테스트

### 📝 다음 단계

1. **FastAPI 서버 실행**
   ```bash
   # 터미널에서 실행
   uvicorn app.main:app --reload --port 8000
   ```

2. **Docker 빌드 및 배포**
   ```bash
   ./scripts/build_and_deploy.sh
   ```

3. **API 테스트**
   - `lab2-1_api_test.ipynb` Notebook 실행
   - 또는 `./scripts/test_api.sh` 스크립트 실행

### 📚 참고 자료

- [Scikit-learn RandomForest 문서](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [Iris 데이터셋 설명](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
- [FastAPI 공식 문서](https://fastapi.tiangolo.com/)

---

© 2025 현대오토에버 MLOps Training